In [73]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
train_values = pd.read_csv('data/train_values.csv', index_col='building_id')
train_labels = pd.read_csv('data/train_labels.csv', index_col='building_id')
test_values = pd.read_csv('data/test_values.csv', index_col='building_id')
submission_format  = pd.read_csv('data/submission_format.csv', index_col='building_id')

In [75]:
selected_features = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone']

train_values_subset = train_values[selected_features]

In [76]:
train_values_subset = pd.get_dummies(train_values_subset)

In [77]:
train_values_subset.head()

,area_percentage,height_percentage,count_floors_pre_eq,has_superstructure_cement_mortar_stone,foundation_type_h,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t
building_id,,,,,,,,,,,,
802906,6,5,2,0,0,0,1,0,0,0,0,1
28830,8,7,2,0,0,0,1,0,0,0,1,0
94947,5,5,2,0,0,0,1,0,0,0,0,1
590882,6,5,2,0,0,0,1,0,0,0,0,1
201944,8,9,3,0,0,0,1,0,0,0,0,1


In [78]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [79]:
pipe = make_pipeline(StandardScaler(), 
                     RandomForestClassifier(random_state=2018))
pipe

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=2018))])

In [80]:
param_grid = {'randomforestclassifier__n_estimators': [50, 100],
              'randomforestclassifier__min_samples_leaf': [1, 5]}
#gs = GridSearchCV(pipe, param_grid, cv=5)

In [81]:
#gs.fit(train_values_subset, train_labels.values.ravel())

In [82]:
#gs.best_params_

In [83]:
#from sklearn.metrics import f1_score

#in_sample_preds = gs.predict(train_values_subset)
#f1_score(train_labels, in_sample_preds, average='micro')

In [84]:
#len(in_sample_preds)

In [85]:
#in_sample_preds

In [86]:
# TEST SET
test_values_subset = test_values[selected_features]
test_values_subset = pd.get_dummies(test_values_subset)

In [87]:
#predictions = gs.predict(test_values_subset)

In [88]:
#my_submission = pd.DataFrame(data=predictions,
#                             columns=submission_format.columns,
#                             index=submission_format.index)

In [89]:
#my_submission.head()

In [90]:
#my_submission.to_csv('submission.csv')

In [91]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          260601 non-null  int64 
 1   geo_level_2_id                          260601 non-null  int64 
 2   geo_level_3_id                          260601 non-null  int64 
 3   count_floors_pre_eq                     260601 non-null  int64 
 4   age                                     260601 non-null  int64 
 5   area_percentage                         260601 non-null  int64 
 6   height_percentage                       260601 non-null  int64 
 7   land_surface_condition                  260601 non-null  object
 8   foundation_type                         260601 non-null  object
 9   roof_type                               260601 non-null  object
 10  ground_floor_type                       260601 non-

In [119]:
features = ['count_floors_pre_eq',
            'area_percentage',
            'height_percentage',
            'age',
            'foundation_type', 
            'ground_floor_type', 
            'other_floor_type',
            'roof_type',
            'has_superstructure_mud_mortar_stone',
            'has_superstructure_cement_mortar_stone']

train_values_subset = train_values #[features]

#CODIFICACION/MANIPULACION DE LA DATA
train_values_subset = pd.get_dummies(train_values_subset)

import xgboost as xgb

xg_reg = xgb.XGBClassifier(objective='multi:softprob',use_label_encoder=False,num_class=3)

data = train_values_subset

data["damage_grade"] = train_labels["damage_grade"] - 1

X, y = data.iloc[:,:-1],data.iloc[:,-1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

#X_train
#y_train
xg_reg.fit(X_train,y_train)

[18:46:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_class=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [64]:
#result = xg_reg.predict(X_train)

#np.set_printoptions(threshold=np.inf)

#from sklearn.preprocessing import OneHotEncoder

#result


#np.set_printoptions(threshold=1000)

In [112]:
from sklearn.metrics import f1_score

in_sample_preds = xg_reg.predict(X_test)

f1_score(y_test.to_frame(), in_sample_preds, average='micro')

0.7249477178104795

In [120]:
# TEST SET
features = ['count_floors_pre_eq',
            'area_percentage',
            'height_percentage',
            'age',
            'foundation_type', 
            'ground_floor_type', 
            'other_floor_type',
            'roof_type',
            'has_superstructure_mud_mortar_stone',
            'has_superstructure_cement_mortar_stone']

test_values_subset = test_values #[features]
test_values_subset = pd.get_dummies(test_values_subset)

In [123]:
predictions = xg_reg.predict(test_values_subset)
predictions=predictions+1

In [124]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submission.csv')

In [103]:
# Selección de las variables por típo
# ==============================================================================
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector

numeric_cols = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.to_list()

# Transformaciones para las variables numéricas
numeric_transformer = Pipeline(
                        steps=[
                            ('imputer', SimpleImputer(strategy='median')),
                            ('scaler', StandardScaler())
                        ]
                      )


# Transformaciones para las variables categóricas
categorical_transformer = Pipeline(
                            steps=[
                                ('imputer', SimpleImputer(strategy='most_frequent')),
                                ('onehot', OneHotEncoder(handle_unknown='ignore'))
                            ]
                          )

preprocessor = ColumnTransformer(
                    transformers=[
                        ('numeric', numeric_transformer, numeric_cols),
                        ('cat', categorical_transformer, cat_cols)
                    ],
                    remainder='passthrough'
                )

In [ ]:
X_train_prep = preprocessor.fit_transform(X_train)
#X_test_prep  = preprocessor.transform(X_test)
# Convertir el output en dataframe y añadir el nombre de las columnas
# ==============================================================================

encoded_cat = preprocessor.named_transformers_['cat']['onehot']\
              .get_feature_names(cat_cols)
labels = np.concatenate([numeric_cols, encoded_cat])
datos_train_prep = preprocessor.transform(X_train)
datos_train_prep = pd.DataFrame(datos_train_prep, columns=labels)
datos_train_prep.info()